#qgrid - SlickGrid in IPython notebook
---

Qgrid is an IPython extension which uses a javascript library called SlickGrid to render pandas DataFrames within an IPython notebook.  It's being developed for use in [Quantopian's hosted research environment]( https://www.quantopian.com/research?utm_source=github&utm_medium=web&utm_campaign=qgrid-nbviewer), and this notebook demonstrates the current state of the project.

## Overview
* [SlickGrid](https://github.com/mleibman/SlickGrid) is an an advanced javascript grid which allows users to scroll, sort, 
and filter hundreds of thousands of rows with extreme responsiveness.  
* [Pandas](https://github.com/pydata/pandas) is a powerful data analysis / manipulation library for Python, and DataFrames are the primary way of storing and manipulating two-dimensional data in pandas.

[Qgrid](https://github.com/quantopian/qgrid) renders pandas DataFrames as SlickGrids, which enables users to explore the entire contents of a DataFrame using intuitive sorting and filtering controls.  It's designed to be used within IPython notebook, and it's also fully functional when rendered in [nbviewer](http://nbviewer.ipython
.org/github/quantopian/qgrid/blob/master/qgrid_demo.ipynb).

## Installation using 'pip install'
##### Qgrid is a python module so you can install it using pip:

```pip install git+https://github.com/quantopian/qgrid```

##### Import it into your namespace like you would for any other python module:

In [1]:
import qgrid

##### Prepare non-python dependencies by calling "`nbinstall`":

In [2]:
qgrid.nbinstall(overwrite=True)  # copies javascript dependencies to your /nbextensions folder 

##### Set default options by calling "set_defaults" (optional):

In [3]:
qgrid.set_defaults(remote_js=True, precision=4)

## Demo 1 - Rendering a DataFrame returned by Yahoo Finance
##### First, lets create a sample DataFrame using pandas 'get_data_yahoo' function:

In [4]:
import pandas as pd
import numpy as np
randn = np.random.randn

pd.set_option('display.max_rows', 8)

from pandas.io.data import get_data_yahoo
spy = get_data_yahoo(
    symbols='SPY',
    start=pd.Timestamp('2011-01-01'),
    end=pd.Timestamp('2014-01-01'),
    adjust_price=True,
)

##### BEFORE - Here's IPython's default representation of our 'spy' DataFrame:

In [5]:
spy

,Open,High,Low,Close,Volume,Adj_Ratio
Date,,,,,,
2011-01-03,115.648405,116.460708,114.726575,115.958727,138725200,0.912701
2011-01-04,116.214283,116.250792,115.173803,115.894838,137409700,0.912701
2011-01-05,115.529756,116.570235,115.420229,116.497217,133975300,0.912701
2011-01-06,116.542855,116.670633,115.922218,116.269042,122519000,0.912701
...,...,...,...,...,...,...
2013-12-26,178.160131,178.762625,178.140706,178.665444,63365000,0.971747
2013-12-27,178.898669,178.976396,178.471098,178.655732,61814000,0.971747
2013-12-30,178.675157,178.820928,178.393357,178.626581,56857000,0.971747
2013-12-31,178.869518,179.471996,178.733459,179.471996,86119900,0.971747


##### AFTER - Here's the same 'spy' DataFrame being rendered as a qgrid:

In [6]:
qgrid.show_grid(spy)

##### Configuring your grid using SlickGrid options
By default, columns are stretched to fill the full width of the grid area.  That doesn't work if you have hundreds of columns, so you can change this behavior by using the `forceFitColumns` and `defaultColumnWidth` SlickGrid options.  See https://github.com/mleibman/SlickGrid/wiki/Grid-Options for the full list of SlickGrid options that you can configure:

In [7]:
qgrid.show_grid(spy, grid_options={'forceFitColumns': False, 'defaultColumnWidth': 200})

## Demo 2 - Rendering a sample DataFrame that includes a MultiIndex
##### Now let's create another DataFrame, but let's give it a MultiIndex this time:

In [8]:
tuples = list(zip(*[['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'],
          ['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two']]))


index = pd.MultiIndex.from_tuples(tuples, names=['first', 'second'])
multi_index_df = pd.DataFrame(randn(8, 2), index=index, columns=['A', 'B'])

##### BEFORE - Here's IPython's default representation of our 'multi_index_df' DataFrame:

In [9]:
multi_index_df

A         B
first second                    
bar   one     1.120275 -1.033297
      two    -1.505228 -0.777790
baz   one    -0.104345  0.436111
      two     1.796850  1.027083
foo   one    -0.589994  0.901359
      two    -0.426320  0.786333
qux   one     2.524368  0.824922
      two    -0.003599 -0.300400

##### AFTER - Here's the same 'multi_index_df' DataFrame being rendered as a qgrid:

In [10]:
qgrid.show_grid(multi_index_df)

## API & Usage
API documentation is hosted on readthedocs: http://qgrid.readthedocs.org/en/latest/

The API documentation can also be accessed via the "?" operator in IPython.  To use the "?" operator, type the name of the function followed by "?" to see the documentation for that function, like this:
```
qgrid.nbinstall?
qgrid.show_grid?
qgrid.set_defaults?
qgrid.set_grid_options?

```

## Running from source using 'git clone'
##### Use git to clone the qgrid repository to your hard drive
`git clone git@github.com:quantopian/qgrid.git`
##### Set the root folder of your qgrid repository to be the working directory for your IPython notebook server
This is an easy way to get qgrid to run directly from the source code.  IPython notebook will look in it's working directory for modules to load, so qgrid will immediately become available for importing.

`ipython notebook --notebook-dir=~/path/to/qgrid/repo`

It's not always convenient to use the qgrid repository as your working directory for IPython noteboook, so I often create symbolic-links from qgrid's source code into the "/nbextensions" and "/extensions" folders under my IPython directory.


##### Import it into your namespace like you would for any other python module:

In [ ]:
import qgrid

##### Prepare non-python dependencies by calling 'nbinstall':

In [ ]:
qgrid.nbinstall(overwrite=True)  # use overwrite=True to keep your /nbextensions folder up to date during development